In [1]:
import pandas as pd
import datetime as dt
import csv

In [2]:
# Read the stock price data from a CSV file
stock_price_all_day = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/stock_price_all_day.csv",
 usecols=["Stock_Code","Date","Trade_Volume","Trade_Value","Opening_Price","Highest_Price","Lowest_Price","Closing_Price","Change","Transaction"],
 encoding='utf-8')

In [3]:
# Calculate the moving averages (MA) using a specified period
def moving_average(data,period):
    return data["Closing_Price"].rolling(period).mean()

In [4]:
# Add moving average columns to the DataFrame
stock_price_all_day["moving_average_5"] = moving_average(stock_price_all_day,5)
stock_price_all_day["moving_average_10"] = moving_average(stock_price_all_day,10)
stock_price_all_day

,Stock_Code,Date,Trade_Volume,Trade_Value,Opening_Price,Highest_Price,Lowest_Price,Closing_Price,Change,Transaction,moving_average_5,moving_average_10
0,2002,2012-1-2,6819530.0,195212661.0,28.80,28.85,28.50,28.55,-0.25,2737.0,NaN,NaN
1,2002,2012-1-3,22624652.0,654596058.0,28.80,29.00,28.70,29.00,0.45,6031.0,NaN,NaN
2,2002,2012-1-4,16452869.0,476519401.0,29.00,29.10,28.80,29.00,0,4604.0,NaN,NaN
3,2002,2012-1-5,17206110.0,497876490.0,28.90,29.00,28.85,29.00,0,3923.0,NaN,NaN
4,2002,2012-1-6,14229626.0,411580354.0,29.00,29.00,28.80,29.00,0,3698.0,28.91,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
225711,2353,2017-1-18,10280315.0,144150610.0,14.00,14.15,13.90,14.05,0,2752.0,13.70,13.530
225712,2353,2017-1-19,13765714.0,194438697.0,14.05,14.25,13.95,14.20,0.15,3654.0,13.84,13.615
225713,2353,2017-1-20,19397952.0,281429076.0,14.20,14.70,14.15,14.50,0.3,5228.0,14.05,13.720
225714,2353,2017-1-23,14742665.0,215420885.0,14.65,14.70,14.50,14.65,0.15,3768.0,14.29,13.855


In [5]:
# Calculate the KD lines
#Step 1: Calculate RSV - (Today's closing price - Lowest price in the past 9 days) / (Highest price in the past 9 days - Lowest price in the past 9 days)
#Step 2: Calculate K - K = 2/3 * (Yesterday's K value) + 1/3 * (Today's RSV)
#Step 3: Calculate D - D = 2/3 * (Yesterday's D value) + 1/3 * (Today's K)

def KD(data):
    # Create a copy of the data DataFrame
    data_df = data.copy()
    
    # Calculate the minimum and maximum values for the past 9 days
    data_df['min'] = data_df["Lowest_Price"].rolling(9).min()
    data_df['max'] = data_df["Highest_Price"].rolling(9).max()
    
    # Calculate RSV
    data_df['RSV'] = (data_df["Closing_Price"] - data_df['min'])/(data_df['max'] - data_df['min'])
    
    # Drop rows with NaN values
    data_df = data_df.dropna()
    
    # Calculate K values
    # Initialize K with a value of 50
    K_list = [50]
    for num,rsv in enumerate(list(data_df['RSV'])):
        K_yestarday = K_list[num]
        K_today = 2/3 * K_yestarday + 1/3 * rsv
        K_list.append(K_today)
    data_df['K'] = K_list[1:]
    
    # Calculate D values
    # Initialize D with a value of 50
    D_list = [50]
    for num,K in enumerate(list(data_df['K'])):
        D_yestarday = D_list[num]
        D_today = 2/3 * D_yestarday + 1/3 * K
        D_list.append(D_today)
    data_df['D'] = D_list[1:]
    
    # Merge the calculated K and D values with the original DataFrame
    use_df = pd.merge(data,data_df[['K','D']],left_index=True,right_index=True,how='left')
    return use_df 

In [6]:
# Apply the KD function to the stock_price_all_day DataFrame
stock_indicator_summary = KD(stock_price_all_day)

In [7]:
# Check for null values in the DataFrame
stock_indicator_summary.isnull()

,Stock_Code,Date,Trade_Volume,Trade_Value,Opening_Price,Highest_Price,Lowest_Price,Closing_Price,Change,Transaction,moving_average_5,moving_average_10,K,D
0,False,False,False,False,False,False,False,False,False,False,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225711,False,False,False,False,False,False,False,False,False,False,False,False,False,False
225712,False,False,False,False,False,False,False,False,False,False,False,False,False,False
225713,False,False,False,False,False,False,False,False,False,False,False,False,False,False
225714,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [8]:
# Fill any null values with 0
stock_indicator_summary = stock_indicator_summary.fillna(value=0)

In [9]:
# Print the updated DataFrame
stock_indicator_summary

,Stock_Code,Date,Trade_Volume,Trade_Value,Opening_Price,Highest_Price,Lowest_Price,Closing_Price,Change,Transaction,moving_average_5,moving_average_10,K,D
0,2002,2012-1-2,6819530.0,195212661.0,28.80,28.85,28.50,28.55,-0.25,2737.0,0.00,0.000,0.000000,0.000000
1,2002,2012-1-3,22624652.0,654596058.0,28.80,29.00,28.70,29.00,0.45,6031.0,0.00,0.000,0.000000,0.000000
2,2002,2012-1-4,16452869.0,476519401.0,29.00,29.10,28.80,29.00,0,4604.0,0.00,0.000,0.000000,0.000000
3,2002,2012-1-5,17206110.0,497876490.0,28.90,29.00,28.85,29.00,0,3923.0,0.00,0.000,0.000000,0.000000
4,2002,2012-1-6,14229626.0,411580354.0,29.00,29.00,28.80,29.00,0,3698.0,28.91,0.000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225711,2353,2017-1-18,10280315.0,144150610.0,14.00,14.15,13.90,14.05,0,2752.0,13.70,13.530,0.731322,0.499760
225712,2353,2017-1-19,13765714.0,194438697.0,14.05,14.25,13.95,14.20,0.15,3654.0,13.84,13.615,0.805008,0.601509
225713,2353,2017-1-20,19397952.0,281429076.0,14.20,14.70,14.15,14.50,0.3,5228.0,14.05,13.720,0.825561,0.676193
225714,2353,2017-1-23,14742665.0,215420885.0,14.65,14.70,14.50,14.65,0.15,3768.0,14.29,13.855,0.872596,0.741661


In [10]:
stock_indicator_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225716 entries, 0 to 225715
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Stock_Code         225716 non-null  int64  
 1   Date               225716 non-null  object 
 2   Trade_Volume       225716 non-null  float64
 3   Trade_Value        225716 non-null  float64
 4   Opening_Price      225716 non-null  float64
 5   Highest_Price      225716 non-null  float64
 6   Lowest_Price       225716 non-null  float64
 7   Closing_Price      225716 non-null  float64
 8   Change             225716 non-null  object 
 9   Transaction        225716 non-null  float64
 10  moving_average_5   225716 non-null  float64
 11  moving_average_10  225716 non-null  float64
 12  K                  225716 non-null  float64
 13  D                  225716 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 24.1+ MB


In [11]:
stock_indicator_summary.to_csv("/Users/elaine/Desktop/side_project/stock_price/stock_indicator_summary.csv.csv", index=False)